<a href="https://colab.research.google.com/github/viswavich/Hexaware_AQB_Phase-1/blob/main/AQB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask llama-index chromadb transformers torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.7 MB/s eta 0:

In [3]:
pip install "pymongo[srv]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.8 MB/s eta 0:00:00


In [5]:
!pip install --upgrade llama-index

In [6]:
import llama_index
print(dir(llama_index))

['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [13]:
import os

# Create the 'documents' directory if it doesn't exist
os.makedirs('documents', exist_ok=True)

# Create a sample text file in the 'documents' directory
with open('documents/sample1.txt', 'w') as f:
    f.write("This is a sample document for testing.")

with open('documents/sample2.txt', 'w') as f:
    f.write("This is another sample document for testing.")


In [16]:
from flask import Flask, request, jsonify
import openai
import chromadb
from chromadb.config import Settings
from pymongo import MongoClient
import torch
from transformers import AutoTokenizer, AutoModel

# Enable GPU acceleration (CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Flask app
app = Flask(__name__)

# Initialize MongoDB Atlas client
client = MongoClient('your-mongodb-atlas-connection-string')
db = client['question_builder_db']
collection = db['questions']

# Initialize Hugging Face API key (if applicable)
hugging_face_api_key = 'your-hugging-face-api-key'

# Initialize Chroma client
chroma_client = chromadb.Client(Settings())

# Check if the collection exists and create it if not
collection_name = "vector_embeddings"
try:
    vector_collection = chroma_client.create_collection(name=collection_name)
except Exception as e:
    if "already exists" in str(e):
        vector_collection = chroma_client.get_collection(name=collection_name)
    else:
        raise e

# Initialize the Hugging Face model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

# Function to generate embeddings using Hugging Face model
def generate_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().detach().numpy()
    return embedding.tolist()  # Convert ndarray to list

# Manually load documents from a directory
def load_documents(directory):
    documents = []
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Directory '{directory}' does not exist.")

    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            with open(filepath, 'r', encoding='utf-8') as file:
                text = file.read()
                documents.append({'id': filename, 'text': text})
    return documents

# Ensure the 'documents' directory exists and contains files
os.makedirs('documents', exist_ok=True)

# Load and process documents
documents = load_documents('documents')
for doc in documents:
    embedding = generate_embedding(doc['text'])
    vector_collection.add(
        embeddings=[embedding],  # Now embedding is a list
        metadatas=[{"text": doc['text']}],  # Pass as a list of dictionaries
        ids=[doc['id']]  # Pass as a list
    )

# Helper function to find similar documents based on a query
def search_chroma(query):
    query_embedding = generate_embedding(query)
    results = vector_collection.query(
        query_embeddings=[query_embedding],
        n_results=5
    )

    relevant_docs = [result['metadata']['text'] for result in results]
    return relevant_docs

# Route to generate a question based on a topic
@app.route('/generate-question', methods=['POST'])
def generate_question_route():
    data = request.json
    topic = data.get('topic')
    difficulty = data.get('difficulty', 'medium')

    if not topic:
        return jsonify({"error": "Topic is required"}), 400

    # Search for relevant documents using Chroma
    relevant_docs = search_chroma(topic)

    # Generate a question using a prompt
    prompt = f"Based on the following information, create a {difficulty} question:\n\n"
    for doc in relevant_docs:
        prompt += f"{doc}\n\n"

    question = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    ).choices[0].text.strip()

    # Store question in MongoDB
    collection.insert_one({"topic": topic, "difficulty": difficulty, "question": question})

    return jsonify({"question": question})

if __name__ == '__main__':
    app.run(debug=True, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [18]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok

run_with_ngrok(app)  # Start ngrok when the app runs
app.run()


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/l

In [21]:
!pip install flask
!pip install "pymongo[srv]==3.3"
!pip install torch torchvision torchaudio
!pip install transformers
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.7/494.7 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymongo: filename=pymongo-3.3.0-cp310-cp310-linux_x86_64.whl size=346801 sha256=2db10fe60e959dce2d7bc752253574285fd0b9f6857660e513ce2b8373513435
  Stored in directory: /root/.cache/pip/wheels/9a/44/8d/cd555bd0bb5714d5fff4c96ea259b3cf0052c7a5f7e6914652
Successfully built pymongo
  Attempting uninstall: pymongo
    Found existing installation: pymongo 4.8.0
    Uninstalling pymongo-4.8.0:
      Successfully uninstalled pymongo-4.8.0
